# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [5]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

# Create an SQLite engine
engine = create_engine('sqlite:///samtable.db')

# Load dataset from database with read_sql_table
df = pd.read_sql_table('samtable', engine)
df.head()

# Define feature and target variables X and Y
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis=1)



### 2. Write a tokenization function to process your text data

In [6]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    
    clean_tokens = [lemmatizer.lemmatize(token).lower().strip() for token in tokens if token not in stop_words]

    return clean_tokens


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# Check data types of target variables
print("Data type of Y_train:", Y_train.dtypes)
print("Data type of Y_test:", Y_test.dtypes)

# Check unique values in target variables
print("Unique values in Y_train:", Y_train.nunique())
print("Unique values in Y_test:", Y_test.nunique())



Data type of Y_train: 2                         float64
related                    object
request                    object
offer                      object
aid_related                object
medical_help               object
medical_products           object
search_and_rescue          object
security                   object
military                   object
child_alone                object
water                      object
food                       object
shelter                    object
clothing                   object
money                      object
missing_people             object
refugees                   object
death                      object
other_aid                  object
infrastructure_related     object
transport                  object
buildings                  object
electricity                object
tools                      object
hospitals                  object
shops                      object
aid_centers                object
other_infrastructure      

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
import nltk
nltk.download('wordnet')

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

pipeline.fit(X_train, Y_train)


[nltk_data] Downloading package wordnet to /root/nltk_data...


TypeError: '>' not supported between instances of 'str' and 'float'

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Fit the pipeline on the training data
pipeline.fit(X_train, Y_train)

# Make predictions on the testing data
Y_pred = pipeline.predict(X_test)

# Report the f1 score, precision, and recall for each output category
for i, category in enumerate(Y.columns):
    print(f"Category: {category}")
    print(classification_report(Y_test[category], Y_pred[:, i]))


TypeError: '>' not supported between instances of 'str' and 'float'

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [10, 50],
    'clf__estimator__min_samples_split': [2, 5, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2, n_jobs=-1)
cv.fit(X_train, Y_train)


TypeError: '>' not supported between instances of 'str' and 'float'

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
Y_pred_tuned = cv.predict(X_test)

# Show the accuracy, precision, and recall of the tuned model
for i, category in enumerate(Y.columns):
    print(f"Category: {category}")
    print(classification_report(Y_test[category], Y_pred_tuned[:, i]))


NotFittedError: This GridSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [10]:
import joblib

joblib.dump(cv.best_estimator_, 'disaster_response_model.pkl')


NameError: name 'cv' is not defined

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.